In [1]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf
# from read_untils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/09/20 09:57:16 WARN Utils: Your hostname, SukiXuudeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 100.71.5.183 instead (on interface en0)
22/09/20 09:57:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/20 09:57:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/20 09:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/20 09:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/20 09:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/09/20 09:57:19 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
import re
import pandas as pd

In [5]:
PLOT_PATH = '../plots/'

In [6]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'tbl_merchants.parquet', truncate=20)

================ Merchant ================
|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------
 name         | Felis Limited        
 tags         | ((furniture, home... 
 merchant_abn | 10023283211          
only showing top 1 row



In [7]:
print("================ Transactions ================")
transactions_sdf = read_file(spark, 'transactions_20210228_20210827_snapshot')

================ Transactions ================
|> Loading File...


|> Loading Finished!
-RECORD 0----------------------------------------------
 user_id        | 18478                                
 merchant_abn   | 62191208634                          
 dollar_value   | 63.255848959735246                   
 order_id       | 949a63c8-29f7-4ab0-ada4-99ac50a88952 
 order_datetime | 2021-08-20                           
only showing top 1 row



### 1. 根据 Transaction 数据可以推测：商家是使用他们的 ABN 作为标识来对应 Transaction 数据


- 由于数量不大，使用 pandas 对与商家的 Tags 数据进行预处理


In [8]:
merchants_sdf.count()

4026

In [9]:
merchants_df_origin = pd.DataFrame(merchants_sdf.collect(), columns=['merchant_name', 'tags', 'merchant_abn']) 
merchants_df = merchants_df_origin.copy()

In [10]:
merchants_df_origin.tags.iloc[0]

'((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))'

- 查看商家和商家 abn 是不是一一对应的 (是一一对应)


In [11]:
# Is there have duplicated merchant abn
merchants_df.loc[merchants_df.duplicated(['merchant_abn'])].size

0

In [12]:
# Is there have duplicated merchants name
merchants_df.loc[merchants_df.duplicated(['merchant_name'])].size

0

### 2. 处理 tags 文本信息


In [13]:
merchants_df = merchants_df_origin.copy()

In [14]:
for i in range(10):
    print(merchants_df.tags.loc[i])

((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))
([cable, satellite, and otHer pay television and radio services], [b], [take rate: 4.22])
([jewelry, watch, clock, and silverware shops], [b], [take rate: 4.40])
([wAtch, clock, and jewelry repair shops], [b], [take rate: 3.29])
([music shops - musical instruments, pianos, and sheet music], [a], [take rate: 6.33])
[(gift, card, novelty, and souvenir shops), (a), (take rate: 6.34)]
[(computers, comPUter peripheral equipment, and softwAre), (b), (take rate: 4.32)]
[[watch, clock, and jewelry repair shops], [c], [take rate: 2.39]]
([computer programming , data processing, and integrated systems design services], [a], [take rate: 5.71])
[(furniture, home furnishings and equipment shopS, and manufacturers, except appliances), (a), (take rate: 6.61)]


In [15]:
# 把tags里的信息，划分成三个column

def tags_reg_func(s):
    s = re.sub(r'\(|\[', '', s[:-2])
    return re.split(r'\), |\], ', s)

merchants_df.tags = merchants_df.tags.apply(lambda x: tags_reg_func(x))

In [16]:
merchants_df['take_rate']  = merchants_df.tags.apply(lambda x: float(re.findall(r"\d+\.?\d*", x[2])[0]) )

In [17]:
merchants_df['type']  = merchants_df.tags.apply(lambda x: x[1] )

In [18]:
# tags信息全小写
merchants_df['tags']  = merchants_df.tags.apply(lambda x: x[0].lower() )

In [ ]:
from collections import defaultdict
new_dict = defaultdict(int)

In [ ]:
merchants_df.tags = merchants_df.tags.apply(lambda x: re.sub(r'\ +', ' ', x))

In [307]:
merchants_df.to_parquet('../data/curated/merchants_data.parquet')

### PLOT

- Sunburst


In [308]:
merchants_df = pd.read_parquet('../data/curated/merchants_data.parquet')

In [309]:
merchants_tag = merchants_df.drop_duplicates('tags').reset_index(drop=True)

In [310]:
# 缩短标签画图
merchants_df_simplised = merchants_df.copy()
# merchants_df_simplised.tags = merchants_df.tags.apply(lambda x: re.sub(r'[^(a-z)]', '', x.split(' ')[0]) )
merchants_df_simplised.tags = merchants_df.tags.apply(lambda x: x.split(',')[0] )
merchants_df_simplised.merchant_name = merchants_df.merchant_name.apply(lambda x: x.split(' ')[0])

In [311]:
# dict_data = defaultdict()
# groupby_tag = merchants_df_simplised.groupby('tags')
# for head, f in groupby_tag:
#     children = defaultdict()
#     grouped = f.groupby('type')
#     for a, b in grouped:
#         child = list(f.groupby('merchant_name').size().reset_index().rename({'merchant_name': 'name', 0:'value'}, axis=1).T.to_dict().values())
#         children[a] = { 'name':a, 'children': child }
#     dict_data[head] = { 'name':head, 'children': list(children.values()) }


In [312]:
# dict_data = defaultdict()
# groupby_tag = merchants_df_simplised.groupby('type')
# for head, f in groupby_tag:
#     child = list(f.groupby('tags').size().reset_index().rename({'tags': 'name', 0:'value'}, axis=1).T.to_dict().values())
#     dict_data[head] = { 'name':head, 'children': child }

In [313]:
dict_data = defaultdict()
groupby_tag = merchants_df_simplised.groupby('tags')
for head, f in groupby_tag:
    child = list(f.groupby('type').size().reset_index().rename({'type': 'name', 0:'value'}, axis=1).T.to_dict().values())
    dict_data[head] = { 'name':head, 'children': child }

In [314]:
from pyecharts.charts import Sunburst
from pyecharts import options as opts

"""
Gallery 使用 pyecharts 1.2.1
参考地址: https://www.echartsjs.com/examples/editor.html?c=sunburst-simple

目前无法实现的功能:

1、暂无
"""
data = list(dict_data.values())
c = (
    Sunburst(init_opts=opts.InitOpts(width="1000px", height="900px"))
    .add(
        "",
        data_pair=data,
        highlight_policy="ancestor",
        radius=[0, "95%"],
        sort_="null",
        levels=[
            {},
            {
                "r0": "10%",
                "r": "60%",
                "itemStyle": {"borderWidth": 2},
                "label": {
                    # "rotate": 'tangential',
                }
            },
            {"r0": "60%", "r": "70%", "label": {
                "align": "right"
                }
            },
            {
                "r0": "70%",
                "r": "72%",
                "label": {"position": "outside", "padding": 3, "silent": False},
                "itemStyle": {"borderWidth": 3},
            },
        ],
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Merchants Tags --- Sunburst"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}"))
    .render(f'{PLOT_PATH}merchants_sunburst.html')
)


- Scatter


In [315]:
merchants_df = pd.read_parquet('../data/curated/merchants_data.parquet')

In [316]:
merchants_df = merchants_df.sort_values('type', axis=0)

In [317]:
data = [list(merchants_df.columns)] + merchants_df.to_numpy().tolist() 

In [321]:
from pyecharts import options as opts
from pyecharts.charts import Grid, Scatter
from pyecharts.globals import ThemeType

l1_1 = (
    Scatter()
    .add_dataset(
        dimensions=[
            "merchant_name",
            "tags",
            "merchant_abn",
            "take_rate",
            {"name": "type", "type": "ordinal"},
        ],
        source=[list(merchants_df.columns)] + merchants_df.to_numpy().tolist() 
    )
    .add_yaxis(
        series_name="",
        y_axis=[],
        symbol_size=2.5,
        xaxis_index=1,
        yaxis_index=1,
        encode={"x": "type", "y": "take_rate", "tooltip": [0, 1, 2, 3]}, 
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="category",
            grid_index=0,
            name="Type",
            boundary_gap=True,
            axislabel_opts=opts.LabelOpts(interval=0),
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value", 
            name="Rate",
        ),
        title_opts=opts.TitleOpts(title="Encode and Matrix"),
    )
    
)




l1_2 = (
    Scatter()
    .add_dataset()
    .add_yaxis(
        series_name="",
        y_axis=[],
        symbol_size=2.5,
        xaxis_index=1,
        yaxis_index=1,
        encode={"x": "take_rate", "y": "tags", "tooltip": [0, 1, 2, 3]}, 
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            type_="category",
            grid_index=1,
            name="Type",
            boundary_gap=True,
            axislabel_opts=opts.LabelOpts(rotate=10, interval=0),
        ),
        xaxis_opts=opts.AxisOpts(
            type_="value", 
            name="Rate",
        ),
    )
)


grid = (
    # theme: CHALK, ESSOS
    Grid(init_opts=opts.InitOpts(width="1960px", height="960px", theme=ThemeType.CHALK))
    .add(
        chart=l1_1,
        grid_opts=opts.GridOpts(pos_left="80%"),
        grid_index=0,
    )
    .add(
        chart=l1_2,
        grid_opts=opts.GridOpts(pos_left="28%", pos_right="25%"),
        grid_index=1,
    )
    .render(f'{PLOT_PATH}merchants_scatter.html')
)

- 

In [295]:
Faker.choose(), Faker.values()

(['小米', '三星', '华为', '苹果', '魅族', 'VIVO', 'OPPO'],
 [68, 29, 102, 47, 28, 118, 94])

In [302]:
merchants_df.groupby('tags').count()

,merchant_name,merchant_abn,take_rate,type
tags,,,,
"antique shops - sales, repairs, and restoration services",3,3,3,3
"antique shops - sales, repairs, and restoration services",2,2,2,2
"antique shops - sales, repairs, and restoration services",124,124,124,124
art dealers and galleries,1,1,1,1
art dealers and galleries,111,111,111,111
...,...,...,...,...
"watch, clock, and jewelry repair shops",2,2,2,2
"watch, clock, and jewelry repair shops",4,4,4,4
"watch, clock, and jewelry repair shops",1,1,1,1


In [293]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line
from pyecharts.faker import Faker

bar = (
    Bar()
    .add_xaxis(Faker.choose())
    .add_yaxis("商家A", Faker.values())
    .add_yaxis("商家B", Faker.values())
    .set_global_opts(title_opts=opts.TitleOpts(title="Grid-Bar"))
)
line = (
    Line()
    .add_xaxis(Faker.choose())
    .add_yaxis("商家A", Faker.values())
    .add_yaxis("商家B", Faker.values())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Grid-Line", pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)

grid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(line, grid_opts=opts.GridOpts(pos_top="60%"))
    .render(f'{PLOT_PATH}merchants_com.html'))